<a href="https://colab.research.google.com/github/edmarja/projects-datascience/blob/main/case_dengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Escenario:**

Es el 1 de enero de 2020. Tu ciudad, históricamente demasiado fría para que sobreviva el mosquito Aedes aegypti, ha comenzado a ver un crecimiento en la población de mosquitos. Por primera vez en la historia de tu ciudad se observa transmisión local del virus del dengue. Eres analista de datos en el departamento de salud pública de la ciudad. La ciudad necesita tener una idea de cuánta transmisión se ha producido y el estado actual de las personas expuestas. Han empezado a hacer pruebas a todos aquellos a los que un mosquito les ha picado recientemente. Tienes la siguiente información sobre cada una de estas personas:

* birthdate — fecha de nacimiento del paciente
* exposed_date — fecha en que el paciente notó por primera vez las picaduras de mosquitos
* test_results — 'Pos' = positivo, 'Neg' = negativo, o 'TBD' = pendiente de resultado (del inglés "to be determined")

Con esta información, clasificarás a cada paciente en una de las siguientes categorías de transmisión y compararás el tamaño de cada categoría entre los grupos de edad.

Las categorías de transmisión son:

* 'Susceptible': no tiene el virus pero puede contraerlo
* 'Exposed': ha sido picado por mosquitos infecciosos y podría tener el virus
* 'Infectious': tiene el virus y actualmente puede transmitirlo a nuevos mosquitos
* 'Recovered': tenía el virus y ya no es infeccioso ni susceptible
* 'Unknown': no hay suficiente información

## **Para el análisis haremos lo siguiente**

* Primero, usaremos la fecha de nacimiento de cada paciente para calcular su edad y clasificarlos en grupos de edad.
* Luego, usaremos exposed_date y test_results para clasificar a los pacientes en una de las cinco categorías de transmisión.
* Finalmente, examinaremos los recuentos dentro de cada categoría de transmisión por grupo de edad utilizando una tabla dinámica.

In [1]:
import pandas as pd

In [2]:
test_data = pd.read_excel('/content/test_results.xlsx', sheet_name=0) #sheet_name=0 llamamos a la primera hoja del excel
print(test_data.info())
print()
test_data.head(10)

#Parece que tenemos dos variables datetime y una variable string. Los tipos de datos parecen estar bien

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   birthdate     786 non-null    datetime64[ns]
 1   exposed_date  786 non-null    datetime64[ns]
 2   test_results  786 non-null    object        
dtypes: datetime64[ns](2), object(1)
memory usage: 18.5+ KB
None



,birthdate,exposed_date,test_results
0,1992-01-08,2019-12-24,Neg
1,1972-01-13,2019-12-15,Neg
2,1981-01-10,2019-12-21,Neg
3,1962-01-15,2019-12-15,Pos
4,1962-01-15,2019-12-04,Pos
5,1989-01-08,2019-12-11,Neg
6,1989-01-08,2019-12-30,TBD
7,1960-01-16,2019-12-24,Pos
8,1942-01-20,2019-12-01,TBD
9,1953-01-17,2019-12-09,Neg


In [ ]:
test_data.isnull().sum() #calculamos si hay valores nulos en cada columna

birthdate       0
exposed_date    0
test_results    0
dtype: int64

In [ ]:
test_data.duplicated().sum() 
#observamos que hay 44 valores duplicados sin embargo no podemos eliminar las filas duplicadas puesto que no sabemos 
# si los resultados corresponden al mismo paciente ya que no contamos con un ID único para cada paciente.

44

In [ ]:
#crearemos una columna 'age' donde mostraremos la edad

today = pd.to_datetime('01-01-2020', format='%d-%m-%Y') #la fecha del estudio '01-01-2020' lo pasamos a formato datetime

def calc_age(birthdate):
    age = today.year - birthdate.year #extraemos el año de cada fecha y las restamos para calcular la edad

    if age <0: #si la edad es menor que cero la edad será nan (nulo)
        return float('nan')
    else:
        return age

test_data['age'] = test_data['birthdate'].apply(calc_age) #apply lee fila por fila 

test_data.head(10)

#La edad se puede calcular con mayor precisión dividiendo el número total de días desde el nacimiento entre 365, 
# pero nuestra estimación aproximada es suficientemente buena por ahora.

,birthdate,exposed_date,test_results,age
0,1992-01-08,2019-12-24,Neg,28
1,1972-01-13,2019-12-15,Neg,48
2,1981-01-10,2019-12-21,Neg,39
3,1962-01-15,2019-12-15,Pos,58
4,1962-01-15,2019-12-04,Pos,58
5,1989-01-08,2019-12-11,Neg,31
6,1989-01-08,2019-12-30,TBD,31
7,1960-01-16,2019-12-24,Pos,60
8,1942-01-20,2019-12-01,TBD,78
9,1953-01-17,2019-12-09,Neg,67


In [ ]:
#para poder agrupar los datos por grupo de edad, necesitamos saber como hacerlo, para ello mostramos los recuentos de edad
#para cada valor único

test_data['age'].value_counts()

#Parece que tenemos muchos pacientes de mediana edad y menos pacientes mayores.

26    24
48    23
27    23
35    23
45    23
      ..
63     2
88     2
90     2
69     2
76     1
Name: age, Length: 74, dtype: int64

In [ ]:
# con sort_index ordenamos los recuentos por edad de menor a mayor, si queremos de mayor a menor pasamos
# dentro de sort_index ascending=False

test_data['age'].value_counts().sort_index() 

#La edad varía de los 15 a los 90 años, con más pacientes pertenecientes a los primeros grupos de edad 
# que a los últimos. Será útil para definir nuestros grupos de edad.

15     8
16     9
17     8
18    10
19    12
      ..
84     3
85     3
86     3
88     2
90     2
Name: age, Length: 74, dtype: int64

Ahora que entendemos la distribución por edades, las clasificamos por rangos de edad.

In [ ]:
# crearemos una función para definir los rangos de edad y agregar una columna al data frame usando apply

def assign_age_group(age):
    if age < 0 or pd.isna(age): #devolvemos NA si la edad en el dataframe es menor a cero o es nan
        return 'NA'
    elif age <=9:
        return '0-9'
    elif age <=19:
        return '10-19'
    elif age <=29:
        return '20-29'
    elif age <=39:
        return '30-39'
    elif age <=49:
        return '40-49'
    elif age <=59:
        return '50-59'
    elif age <=69:
        return '60-69'
    else:
        return '70+'

test_data ['age_group'] = test_data['age'].apply(assign_age_group)

test_data.head(10)

,birthdate,exposed_date,test_results,age,age_group
0,1992-01-08,2019-12-24,Neg,28,20-29
1,1972-01-13,2019-12-15,Neg,48,40-49
2,1981-01-10,2019-12-21,Neg,39,30-39
3,1962-01-15,2019-12-15,Pos,58,50-59
4,1962-01-15,2019-12-04,Pos,58,50-59
5,1989-01-08,2019-12-11,Neg,31,30-39
6,1989-01-08,2019-12-30,TBD,31,30-39
7,1960-01-16,2019-12-24,Pos,60,60-69
8,1942-01-20,2019-12-01,TBD,78,70+
9,1953-01-17,2019-12-09,Neg,67,60-69


***

In [ ]:
#EJEMPLO: también se puede calcular la diferencia entre datetimes

date_diff = today - test_data['exposed_date'].iloc[0] #diferencia entre 01-01-2020 y 24-12-2019

print(date_diff)
print()
print(type(date_diff)) #el tipo de dato es un timedelta donde delta indica diferencia
date_diff.days #el método 'days' entrega el número de días cuando el tipo de dato es un timedelta

8 days 00:00:00

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


8

***

In [ ]:
# Ahora necesitamos extraer información de la variable exposeddate para asignar las categorías de transmisión.

today = pd.to_datetime('01-01-2020', format='%d-%m-%Y') #pasamos la fecha de estudio a datetime

def calc_days_since_exposed(exposed): #creamos la función para extraer los días de la columna 'exposed_date'
    days_diff = today - exposed

    if days_diff.days <0:
        return float('nan')
    else:
        return days_diff.days

test_data['days_since_exposed'] = test_data['exposed_date'].apply(calc_days_since_exposed) #agregamos nueva columna y con apply aplicamos la fucnión

test_data.head(10)



,birthdate,exposed_date,test_results,age,age_group,days_since_exposed
0,1992-01-08,2019-12-24,Neg,28,20-29,8
1,1972-01-13,2019-12-15,Neg,48,40-49,17
2,1981-01-10,2019-12-21,Neg,39,30-39,11
3,1962-01-15,2019-12-15,Pos,58,50-59,17
4,1962-01-15,2019-12-04,Pos,58,50-59,28
5,1989-01-08,2019-12-11,Neg,31,30-39,21
6,1989-01-08,2019-12-30,TBD,31,30-39,2
7,1960-01-16,2019-12-24,Pos,60,60-69,8
8,1942-01-20,2019-12-01,TBD,78,70+,31
9,1953-01-17,2019-12-09,Neg,67,60-69,23


***
### **El panorama general hasta el momento:**

¿Por qué necesitamos el número de días desde la exposición? Necesitamos esta información para determinar si las personas que dan positivo se han recuperado o son infecciosos actualmente. Para la fiebre del dengue, el período infeccioso puede durar entre seis y 17 días tras la exposición. Esto significa que asumimos que los que dan positivo, si su valor de días desde la exposición es 17 o menos, todavía son infecciosos. Si es mayor que 17, asumimos que se han recuperado. El objeto timedelta es muy útil para calcular la diferencia entre fechas, como acabas de ver.
***

In [ ]:
#Es hora de asignar a cada paciente una de las cinco categorías de transmisión. 
#Lo haremos usando sus test_results y la variable derivada days_since_exposed

def assign_status(row): #creamos la función 
    
    test = row['test_results']
    days_exposed = row['days_since_exposed']
    
    if test == 'Neg':
        return 'Susceptible'
    elif test == 'TBD' and days_exposed<=17:
        return 'Exposed'
    elif test == 'Pos' and days_exposed<=17:
        return 'Infectious'
    elif test == 'Pos' and days_exposed>17:
        return 'Recovered'
    elif test == 'TBD' and days_exposed>17:
        return 'Unknown'

#Probamos si la función ejecutra correctamente

row_values = ['Pos', 28]
column_values = ['test_results', 'days_since_exposed']

row = pd.Series(data=row_values, index=column_values)

assign_status(row) #los resultados son correctos

'Recovered'

In [ ]:
# Ahora creamos una nueva columna asignando a cada paciente una categoría de transmisión aplicando assign_status() al conjunto de datos

def assign_status(row): #creamos la función 
    
    test = row['test_results']
    days_exposed = row['days_since_exposed']
    
    if test == 'Neg':
        return 'Susceptible'
    elif test == 'TBD' and days_exposed<=17:
        return 'Exposed'
    elif test == 'Pos' and days_exposed<=17:
        return 'Infectious'
    elif test == 'Pos' and days_exposed>17:
        return 'Recovered'
    elif test == 'TBD' and days_exposed>17:
        return 'Unknown'

test_data['status'] = test_data.apply(assign_status, axis=1) #axis =1 para aplicar a las filas

test_data.head(10)

,birthdate,exposed_date,test_results,age,age_group,days_since_exposed,status
0,1992-01-08,2019-12-24,Neg,28,20-29,8,Susceptible
1,1972-01-13,2019-12-15,Neg,48,40-49,17,Susceptible
2,1981-01-10,2019-12-21,Neg,39,30-39,11,Susceptible
3,1962-01-15,2019-12-15,Pos,58,50-59,17,Infectious
4,1962-01-15,2019-12-04,Pos,58,50-59,28,Recovered
5,1989-01-08,2019-12-11,Neg,31,30-39,21,Susceptible
6,1989-01-08,2019-12-30,TBD,31,30-39,2,Exposed
7,1960-01-16,2019-12-24,Pos,60,60-69,8,Infectious
8,1942-01-20,2019-12-01,TBD,78,70+,31,Unknown
9,1953-01-17,2019-12-09,Neg,67,60-69,23,Susceptible


Ahora usaremos los grupos de edad que hemos creado y las categorías asignadas para examinar el estado actual del dengue en nuestra comunidad.

También usaremos una tabla dinámica para ver los recuentos de categorías (columnas) por grupo de edad (filas). (Incluiremos el argumento margins=True para incluir los totales de fila/columna).

Nota: ya que estamos hablando de los recuentos, usamos el argumento de entrada values='age' para simplificar la tabla.


In [ ]:
#creamos una tabla dinámica para mostrar mejor los resultados:
# la fila será los grupos de edad
# las columnas será las categorías
# agregamos margin = True para que se vea una columna adicional (All) con el total

dinamic_results = test_data.pivot_table(index='age_group', columns='status', values='age', aggfunc='count', margins=True)

dinamic_results.head(10) #nuestra tabla está lista para presentar los resultados

status,Exposed,Infectious,Recovered,Susceptible,Unknown,All
age_group,,,,,,
10-19,3,2,8,22,12,47
20-29,11,14,40,53,26,144
30-39,8,16,31,69,35,159
40-49,7,10,56,65,14,152
50-59,11,10,26,57,12,116
60-69,6,9,18,36,7,76
70+,8,8,23,41,12,92
All,54,69,202,343,118,786


## **Resumen:**

Se puede obtener muchas ideas de la tabla dinámica, que incluyen:
* Hay muchos pacientes susceptibles y recuperados en todos los grupos de edad.
* La mayoría de los que siguen infecciosos tienen de 30 a 39 años.
* En los grupos de edad de 20-29 y 30-39 años muchos pacientes entran en la categoría unknown
* Hay más personas de 40-49 años que se han recuperado que en cualquier otro grupo de edad.

El alto número de pacientes en la categoría unknown debería alentar a la ciudad a obtener los resultados de las pruebas TBD lo más rápido posible. El alto número de pacientes recuperados en todos los grupos de edad es preocupante porque implica que el virus del dengue se transmitió durante un tiempo antes de que se analizaran los resultados de las pruebas. Estos son los datos que tu jefe quiere ver, y solo nos tomó 13 ejercicios para llegar allí.

En este estudio de caso, empezamos con tres columnas y terminamos con siete. Esto no es algo poco frecuente. Las cuatro columnas que creamos fueron necesarias para examinar los datos de la forma en que lo hicimos y el método apply() fue útil y facilitó la creación de estas cuatro columnas.